In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
sns.set(font_scale=1.5)

# データと確認

ラベルとテキストの2要素を持つ。
テキストは日本語文であり、それを形態素解析して機械学習の素性とする。

In [ ]:
## データをファイルから読み込む
# 各行の例: ^Positive[\t]これは素晴らしい。$
#df_tmp = pd.read_csv('data/sample.tsv', sep='\t', names=('Label', 'OriginalText'))

In [ ]:
## データを直接セットする
df_tmp = pd.DataFrame([
    {'Label': 'Positive', 'OriginalText': 'これは素晴らしい。'},
    {'Label': 'Negative', 'OriginalText': '最悪の事態です。'},
    {'Label': 'Other', 'OriginalText': 'お腹がすいた。'},
    {'Label': 'Positive', 'OriginalText': '嬉しいことがあった'},
    {'Label': 'Negative', 'OriginalText': 'こういうのはダメだと思う。'},
    {'Label': 'Other', 'OriginalText': 'ペットボトルのお茶を飲みました。'},
    {'Label': 'Positive', 'OriginalText': 'やったね！'},
    {'Label': 'Negative', 'OriginalText': '悪い結果です。'},
    {'Label': 'Other', 'OriginalText': '中性脂肪をやっつけろ！'},
    {'Label': 'Positive', 'OriginalText': 'すごく良い。素晴らしき世界。'},
    {'Label': 'Negative', 'OriginalText': '最低最悪の事態が発生しました'},
    {'Label': 'Other', 'OriginalText': 'アーモンドとカシューナッツ、どっちにする？'},
    {'Label': 'Positive', 'OriginalText': '大好きなお菓子を買えて嬉しい。'},
    {'Label': 'Negative', 'OriginalText': 'こういうの絶対ダメ。ゆるされない。'},
    {'Label': 'Other', 'OriginalText': '現在の時刻は午後3時30分です'},
    {'Label': 'Positive', 'OriginalText': '最高です。とても良い結果です。'},
    {'Label': 'Negative', 'OriginalText': 'ちょー最悪、ありえない'},
    {'Label': 'Other', 'OriginalText': 'チャイムが鳴りました。'},
    {'Label': 'Positive', 'OriginalText': '今日の天気は晴。久しぶりなので嬉しいな'},
    {'Label': 'Negative', 'OriginalText': 'やめてくれよこういうの。最低。'},
    {'Label': 'Other', 'OriginalText': '雨が降っています。'},
    {'Label': 'Other', 'OriginalText': '夏は暑いが冬は寒い'},
    {'Label': 'Other', 'OriginalText': 'ぬるいコーヒーを飲む。'},
])

In [ ]:
df_tmp.head(3)

In [ ]:
# ラベルの分布の確認
tmp = df_tmp['Label'].value_counts()
print(tmp.to_dict())
tmp.plot.bar()

# テキストの処理

In [ ]:
# テキストから余計な文字列を除去する処理
def clean_sentence(sentence):
    sentence = re.sub('https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', sentence)
    return sentence

# テキストのクリーニング
df_tmp['Text'] = df_tmp['OriginalText'].apply(clean_sentence)
df_tmp.sample(2)

In [ ]:
### 形態素解析 janome
from janome.tokenizer import Tokenizer
janome = Tokenizer()
def ma_sentence(sentence):
    return [{'surface': x.surface, 'pos': x.part_of_speech, 'base_form': x.base_form} for x in janome.tokenize(sentence)]

ma_sentence('菓子を買ってもお金は減らず。')

In [ ]:
### 形態素解析 MeCab
import MeCab
mecab = MeCab.Tagger('-Ochasen')
def ma_sentence(sentence):
    mors = [x.split('\t') for x in mecab.parse(sentence).split('\n')]
    return [{'surface': mor[0], 'pos': mor[3], 'base_form': mor[2]} for mor in mors if len(mor) > 2]

ma_sentence('菓子を買ってもお金は減らず。')

In [ ]:
def get_mor(sentences):
    mors_list = [ma_sentence(x) for x in sentences]
    chunked_sentences = []
    for mors in mors_list:
        ## 特定の品詞の形態素のみ使う
        strs = [mor['base_form'] for mor in mors if re.match('名詞(?!.*(代名詞|接尾|非自立))|形容詞|動詞', mor['pos'])]
        ## 前形態素を使う
        #strs = [mor['surface'] for mor in mors]
        chunked_sentences.append("\t".join(strs))
    return chunked_sentences
print(get_mor(['これは綺麗なペンです。','今日は歩きました'])) # => ['綺麗\tペン', '今日\t歩く']

# テキストを単語分かち書きに
chunks_train = get_mor(df_tmp['Text'])
print(chunks_train[0:-1])

In [ ]:
# 機械学習用の学習データへの変換 ベクトル化
from sklearn.feature_extraction.text import CountVectorizer
vec_train = CountVectorizer(binary=True, ngram_range=(1,1), min_df=1, token_pattern='[^\\t]+')
X_train = vec_train.fit_transform(chunks_train)
vocabulary = vec_train.vocabulary_
print(list(vocabulary.items())[0:20])

In [ ]:
### 文字ベース
# 形態素解析を用いない場合はこちら

# 機械学習用の学習データへの変換 ベクトル化
from sklearn.feature_extraction.text import CountVectorizer
vec_train = CountVectorizer(binary=True, ngram_range=(2,2), min_df=1, token_pattern='.')
X_train = vec_train.fit_transform(df_tmp['Text'])
vocabulary = vec_train.vocabulary_
print(list(vocabulary.items())[0:20])

In [ ]:
y_train = df_tmp['Label']

In [ ]:
print(X_train[0:2])
print("------")
print(y_train[0:2])

# 2次元可視化

可視化することで問題の難易度が分かるかもしれないし、分からないかもしれない。

In [ ]:
# 2次元に変換して図にする関数
def kashika(X, y, X_len=[], random_state=0, n_SVD=2, sampling_rate=0.1):

    # 可視化に使うデータをサンプリング
    from sklearn.utils import shuffle
    shuffled_index = shuffle(np.arange(0, len(y)), random_state=random_state)
    mask = shuffled_index[:int(len(y)*sampling_rate)]
    X_sample = X[mask]
    
    # 2次元に変換
    from sklearn.decomposition import TruncatedSVD
    deco_model = TruncatedSVD(n_components=n_SVD)
    proj = deco_model.fit_transform(X_sample)
    
    # 描画
    fig = plt.figure(figsize=(10,10))
    sns.scatterplot(proj[:,0], proj[:,1],
                    hue=y[mask],
                    alpha=0.5)

In [ ]:
fig = kashika(X_train, y_train, sampling_rate=1)

# 機械学習と精度確認

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import (cross_val_score, cross_val_predict, KFold)
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='auto')

In [ ]:
y_pred = cross_val_predict(logistic, X_train, y_train, cv=KFold(n_splits=2, shuffle=False))

In [ ]:
conf_mat = confusion_matrix(y_train, y_pred)

fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(conf_mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='RdPu',
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))
ax.set_ylim(len(conf_mat), 0) # for matplotlib 3.1.1 以下。3.1.2以降は不要
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

In [ ]:
print(classification_report(y_train, y_pred))
clarep = pd.DataFrame(classification_report(y_train, y_pred, output_dict=True)).T
clarep['support'] = clarep['support'].astype(int)
display(clarep)

# 個別判定結果の確認

In [ ]:
df_tmp['Pred'] = y_pred

In [ ]:
# 正しく判定できた例
df_tmp[df_tmp['Label'] == df_tmp['Pred']][['Label','Pred','OriginalText']].head()

In [ ]:
# 正しく判定できなかった例
df_tmp[df_tmp['Label'] != df_tmp['Pred']][['Label','Pred','OriginalText']].head()

# 判定に寄与する素性の確認

In [ ]:
traall = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='auto')
traall.fit(X_train, y_train)

In [ ]:
fdf = pd.DataFrame(traall.coef_.T, columns=traall.classes_)

In [ ]:
#fdf['vocabulary'] = vec_train.get_feature_names_out()
fdf['vocabulary'] = vec_train.get_feature_names()

In [ ]:
feature_ranking = pd.DataFrame()
for c in traall.classes_:
    p = fdf.sort_values([c], ascending=False)[[c, 'vocabulary']]
    p.index = np.arange(len(p))
    feature_ranking = pd.concat([feature_ranking, p], axis=1)

In [ ]:
feature_ranking[0:10]